    0000000_16_230203_002_final_cluster_happy_parking_map_after
     
    data :  kyungki_region.json(경계), 230128_cluster_map_lat.csv(경계 좌표)
            k_happy_df.csv(행복주택),korea_parking.csv(주차장)
            
    save : 230203_result_after.html
           
    목차 
        0) package load , 선정지역 input
        1) 경기도 시각화 
           - 경계 
           - 경기도 시구 : 42 개 좌표    ( 16개 좌표 만들기 위한 중간 data )
           - 선정 지역 경계 표시 
        2) 행복주택 data/좌표 -> 선정 지역 행복주택 경계 표시 
        3) 전국주차장 data/좌표 > 경기도주차장 좌표 -> 전처리(reset_index) -> 선정 지역 행복주택 경계 표시 
        4) 시각화 
           - base map -> 경기도 경계 -> 선정지역  color map -> 주차장 -> 행복지구 -> 행복주택 원표시 -> total map
        5) save

In [1]:
#0) package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#0) 선정지역 input
list = []
sigu_name= ''
while sigu_name != 'e':
    sigu_name = input("Enter sigu, or 'e': ")

    if sigu_name != 'e':
        list.append(sigu_name)
list

Enter sigu, or 'e': 성남시 수정구
Enter sigu, or 'e': 화성시
Enter sigu, or 'e': e


['성남시 수정구', '화성시']

In [3]:
# 1) 경기도 경계 시각화 
k_geo_data_path = '../002_data_dir/001_map_data/kyungki_region.json'
k_geo_data = json.load(
    open(k_geo_data_path, encoding="utf-8")
)

print('슝~')

슝~


In [4]:
#1) 경기도 시구 : 42 개 좌표 -> 분석 대상 : 경기도 16개 좌표
k_map_df = pd.read_csv("../002_data_dir/003_search_data/230128_cluster_map_lat.csv")
k_16_df = k_map_df[k_map_df['socar_data'] == 1]
k_16_df.loc[:,'adm_cd'] = k_16_df.loc[:,'adm_cd'].astype(str)
k_16_df.head()

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,adm_cd,adm_nm,socar_data,cluster,char,lon,lat,adm_nm1,adm_nm2
1,41281,고양시 덕양구,1,1.0,A,126.878632,37.655810,고양시,덕양구
2,41285,고양시 일산동구,1,1.0,A,126.797558,37.679865,고양시,일산동구
3,41287,고양시 일산서구,1,1.0,A,126.727983,37.680205,고양시,일산서구
5,41210,광명시,1,2.0,B,126.866708,37.475750,광명시,NaN
7,41310,구리시,1,2.0,B,127.131864,37.591625,구리시,NaN


In [5]:
#1) 선정 지역 경계 표시 
k_final_df = k_16_df.copy(deep= True)
temp = len(k_final_df)
k_final_df.drop(k_final_df.index[0:temp], inplace=True)

for i in range(len(list)):
    name_df = k_16_df[k_16_df['adm_nm'] == list[i]]
    if i > 0:
        k_final_df = k_final_df.append(name_df)
    else:
        k_final_df = name_df

k_final_df = k_final_df.reset_index(drop=True)    

k_final_df

,adm_cd,adm_nm,socar_data,cluster,char,lon,lat,adm_nm1,adm_nm2
0,41131,성남시 수정구,1,3.0,C,127.104145,37.435166,성남시,수정구
1,41590,화성시,1,5.0,E,126.833531,37.196817,화성시,NaN


In [6]:
#2) 행복주택 data/좌표
path = '../002_data_dir/003_search_data/happy_house_df1.csv'
happy_house_df = pd.read_csv(path)
happy_house_df.head()  

,지역,단지,세대수,주소,lng,lat
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,126.865491,37.650992
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,126.921037,37.653331
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,126.914946,37.654038
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,126.868888,37.652631
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,126.847648,37.621215


In [7]:
#2) 선정 지역 행복주택 경계 표시 
k_final_happy = happy_house_df.copy(deep= True)
temp = len(k_final_happy)
k_final_happy.drop(k_final_happy.index[0:temp], inplace=True)

for i in range(len(list)):
    name_df = happy_house_df[happy_house_df['지역'] == list[i]]
    if i > 0:
        k_final_happy = k_final_happy.append(name_df)
    else:
        k_final_happy = name_df

k_final_happy = k_final_happy.reset_index(drop=True)    

k_final_happy

,지역,단지,세대수,주소,lng,lat
0,성남시 수정구,성남고등 A-1 행복주택리츠,1040,경기도 성남시 수정구 고등로 57,127.099577,37.430530
1,성남시 수정구,판교제2테크노밸리 A1블록 창업지원주택,200,경기도 성남시 수정구 창업로40번길 6,127.095995,37.412634
2,성남시 수정구,성남위례A2-15BL 지역전략산업 및 창업지원주택,470,경기도 성남시 수정구 위례순환로 53,127.135125,37.471790
3,성남시 수정구,성남단대,16,경기도 성남시 수정구 산성대로405번길 9-9,127.157143,37.448183
4,성남시 수정구,단대동 행복주택,60,경기도 성남시 수정구 단대로23번길 36,127.155863,37.449843
5,화성시,"화성봉담2(LH4단지)(행복,임대상가)",602,경기도 화성시 봉담읍 상리3길 111,126.951197,37.229086
6,화성시,엘크루 루엘리안,152,경기도 화성시 봉담읍 상리2길 121,126.944052,37.229142
7,화성시,"화성남양뉴타운(LH6단지)(행복,임대상가)",410,경기도 화성시 남양읍 남양로862번길 16,126.834273,37.212523
8,화성시,남양뉴타운 B-9블록 행복주택,880,경기도 화성시 남양읍 남양중앙로 420,126.841182,37.202765
9,화성시,화성동탄2 A57-1블록 행복주택,850,경기도 화성시 동탄신리천로8길 15,127.142259,37.175637


In [8]:
#3) 전국주차장 data/좌표 > 경기도주차장 좌표 -> 전처리(reset_index)
paking_station_info_df = pd.read_csv("../002_data_dir/003_search_data/k_parking_area.csv")
k_parking = paking_station_info_df[paking_station_info_df['PROVD_INSTT_NM1'] == '경기도']
k_parking = k_parking[k_parking['FCLTY_LO'].notnull()]
k_parking = k_parking.reset_index(drop=True)
len(k_parking)

1937

In [9]:
#3) 경기도 행복주택 경계 표시
k_final_parking = k_parking.copy(deep= True)
temp = len(k_final_parking)
k_final_parking.drop(k_final_parking.index[0:temp], inplace=True)

for i in range(len(list)):
    name_df = k_parking[k_parking['sigu_name'] == list[i]]
    if i > 0:
        k_final_parking = k_final_parking.append(name_df)
    else:
        k_final_parking = name_df

k_final_parking = k_final_parking.reset_index(drop=True)    

k_final_parking

,ID,LCLAS_NM,MLSFC_NM,FCLTY_NM,CTPRVN_NM,SIGNGU_NM,LEGALDONG_CD,LEGALDONG_NM,ADSTRD_CD,ADSTRD_NM,...,FILE_NM,BASE_DE,PROVD_INSTT_NM1,PROVD_INSTT_NM2,RDNMADR_NM1,RDNMADR_NM2,RDNMADR_NM3,RDNMADR_NM4,RDNMADR_NM5,sigu_name
0,KC490PC22N002112,장소,주차장,성남시청부설주차장,경기도,성남시,NaN,NaN,NaN,NaN,...,KC_490_WNTY_PRKLT_2022,20221117,경기도,성남시,경기도,성남시,성남대로,997,NaN,성남시 수정구
1,KC490PC22N005206,장소,주차장,건우,경기도,성남시 수정구,4113110200,태평동,4.113155e+09,태평2동,...,KC_490_WNTY_PRKLT_2022,20221117,경기도,성남시,경기도,성남시,수정구,태평로,55번길,성남시 수정구
2,KC490PC22N005207,장소,주차장,수진 제1,경기도,성남시 수정구,4113110300,수진동,4.113157e+09,수진1동,...,KC_490_WNTY_PRKLT_2022,20221117,경기도,성남시,경기도,성남시,수정구,제일로,114,성남시 수정구
3,KC490PC22N005208,장소,주차장,수진 제3,경기도,성남시 수정구,4113110300,수진동,4.113157e+09,수진1동,...,KC_490_WNTY_PRKLT_2022,20221117,경기도,성남시,경기도,성남시,수정구,제일로,128,성남시 수정구
4,KC490PC22N005209,장소,주차장,번성,경기도,성남시 수정구,4113110300,수진동,4.113158e+09,수진2동,...,KC_490_WNTY_PRKLT_2022,20221117,경기도,성남시,경기도,성남시,수정구,수정남로,14,성남시 수정구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,KC490PC22N012760,장소,주차장,능동제3,NaN,NaN,NaN,NaN,NaN,NaN,...,KC_490_WNTY_PRKLT_2022,20211020,경기도,화성시,NaN,NaN,NaN,NaN,NaN,화성시
131,KC490PC22N012761,장소,주차장,산척,NaN,NaN,NaN,NaN,NaN,NaN,...,KC_490_WNTY_PRKLT_2022,20211020,경기도,화성시,NaN,NaN,NaN,NaN,NaN,화성시
132,KC490PC22N012762,장소,주차장,석우동IT단지,NaN,NaN,NaN,NaN,NaN,NaN,...,KC_490_WNTY_PRKLT_2022,20211020,경기도,화성시,NaN,NaN,NaN,NaN,NaN,화성시
133,KC490PC22N012763,장소,주차장,동탄2오산동(신리천로),NaN,NaN,NaN,NaN,NaN,NaN,...,KC_490_WNTY_PRKLT_2022,20211020,경기도,화성시,NaN,NaN,NaN,NaN,NaN,화성시


In [10]:
#4) 시각화 : base map 지정

center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 10으로 시작 합니다.
map = folium.Map(
    location=center,
    zoom_start=10,
    )

print('슝~')

슝~


In [11]:
#4) 시각화 : 경기도 경계
folium.GeoJson(
    k_geo_data,
    name='경기도'
).add_to(map)

In [12]:
#4) 시각화 : 선정지역 
folium.Choropleth(
    geo_data=k_geo_data,
    name="cluster map",
    data=k_final_df,
    columns=["adm_cd", "cluster"],
    key_on="feature.properties.ADM_SECT_C",
    fill_color='Accent',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    nan_fill_color='blue',
    nan_fill_opacity =0,
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    legend_name="cluster"
).add_to(map)



In [13]:
#4) 시각화 : 선정지역 주차장
marker_cluster = MarkerCluster().add_to(map)

# 주차장 이름, 위도, 경도를 가져와 maker 객체로 추가
for index_num in range(len(k_final_parking)):
    try:
        folium.Marker(
            location=[k_final_parking.loc[index_num, "FCLTY_LA"], k_final_parking.loc[index_num, "FCLTY_LO"]],
            popup=k_final_parking.loc[index_num, "FCLTY_NM"],
            icon=folium.Icon(color="cadetblue", icon="ok")
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [14]:
#4) 시각화 : 선정지역 행복주택지구 대한 원표시

#maker 객체 추가
for _, row in k_final_happy.iterrows():
  
        folium.CircleMarker(
            location=[row['lat'], row['lng']],
            color='tomato',
            radius=50,
            tooltip = 'zone'           
        ).add_to(map)


print('슝~')

슝~


In [15]:
# #7 시각화 . map
# map

In [16]:
# map.save("../003_map_xml/230203_result_after.html")